In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertModel
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from tqdm import tqdm

In [ ]:
def extract_bert_embeddings(urls):
    """
    Extract embeddings from URLs using BERT.
    """
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    model = BertModel.from_pretrained('bert-base-uncased').to('cuda')
    # model = BertModel.from_pretrained('bert-base-uncased')
    model.eval()

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    url_embeddings = []
    print("Extracting BERT Embeddings...")
    for url in tqdm(urls, desc="Embedding URLs"):
        inputs = tokenizer(url, return_tensors='pt', padding=True, truncation=True, max_length=128).to(device)
        # inputs = tokenizer(url, return_tensors='pt', padding=True, truncation=True, max_length=128)
        with torch.no_grad():
            outputs = model(**inputs)
        embeddings = outputs.last_hidden_state[:, 0, :].squeeze().cpu().numpy()

        # embeddings = outputs.last_hidden_state[:, 0, :].squeeze().numpy()
        url_embeddings.append(embeddings)

    return url_embeddings

In [ ]:
# Step 1: Load Dataset
data = pd.read_csv('/kaggle/input/info-sec-a2-data/balanced_data_with_features.csv')

# Step 2: Train-Test Split
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# Step 3: Extract URL Embeddings for Train and Test Sets
train_urls = train_data['url'].tolist()
test_urls = test_data['url'].tolist()

train_embeddings = extract_bert_embeddings(train_urls)
test_embeddings = extract_bert_embeddings(test_urls)

# Step 4: Prepare Labels
y_train = train_data['label'].values
y_test = test_data['label'].values

# Step 5: Train a Simple Classifier (Logistic Regression)
print("Training the Classifier...")
classifier = LogisticRegression(max_iter=1000)
classifier.fit(train_embeddings, y_train)

# Step 6: Make Predictions and Evaluate Model
print("Evaluating the Model...")
y_pred = classifier.predict(test_embeddings)
report = classification_report(y_test, y_pred)
print(report)

In [ ]:
print("hello")

# trying another classifier

In [2]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertModel
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from tqdm import tqdm

In [3]:
def extract_bert_embeddings(urls):
    """
    Extract embeddings from URLs using BERT with GPU support (if available).
    """
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    model = BertModel.from_pretrained('bert-base-uncased').to(device)
    model.eval()

    url_embeddings = []
    print("Extracting BERT Embeddings...")
    for url in tqdm(urls, desc="Embedding URLs"):
        inputs = tokenizer(url, return_tensors='pt', padding=True, truncation=True, max_length=128).to(device)
        with torch.no_grad():
            outputs = model(**inputs)
        embeddings = outputs.last_hidden_state[:, 0, :].squeeze().cpu().numpy()
        url_embeddings.append(embeddings)

    return url_embeddings

In [4]:
import joblib


def train_and_save_model(model, model_name, train_embeddings, y_train, test_embeddings, y_test):
    """
    Train a given model, print classification report, and save the model weights.
    """
    print(f"\nTraining {model_name}...")
    model.fit(train_embeddings, y_train)
    print(f"Evaluating {model_name}...")
    y_pred = model.predict(test_embeddings)
    report = classification_report(y_test, y_pred)
    print(report)

    # Save the trained model
    model_path = f"{model_name}_model.pkl"
    joblib.dump(model, model_path)
    print(f"{model_name} model saved successfully at {model_path}.")

In [ ]:
# Step 1: Load Dataset
data = pd.read_csv('/kaggle/input/info-sec-a2-data/balanced_data_with_features.csv')

# Step 2: Train-Test Split
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# Step 3: Extract URL Embeddings for Train and Test Sets
train_urls = train_data['url'].tolist()
test_urls = test_data['url'].tolist()

train_embeddings = extract_bert_embeddings(train_urls)
test_embeddings = extract_bert_embeddings(test_urls)

# Step 4: Prepare Labels
y_train = train_data['label'].values
y_test = test_data['label'].values

# Step 5: Train and Save Models Individually
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
train_and_save_model(rf_classifier, 'RandomForest', train_embeddings, y_train, test_embeddings, y_test)

xgb_classifier = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
train_and_save_model(xgb_classifier, 'XGBoost', train_embeddings, y_train, test_embeddings, y_test)

svm_classifier = SVC(kernel='rbf', probability=True)
train_and_save_model(svm_classifier, 'SVM', train_embeddings, y_train, test_embeddings, y_test)

Extracting BERT Embeddings...


Embedding URLs: 100%|██████████| 411972/411972 [55:26<00:00, 123.85it/s] 


Extracting BERT Embeddings...


Embedding URLs: 100%|██████████| 102994/102994 [13:52<00:00, 123.66it/s]



Training RandomForest...
Evaluating RandomForest...
              precision    recall  f1-score   support

           0       0.91      0.89      0.90     18778
           1       0.92      0.95      0.94     20048
           2       0.95      0.98      0.96     18912
           3       0.58      0.61      0.59     23738
           4       0.56      0.51      0.53     21518

    accuracy                           0.77    102994
   macro avg       0.78      0.79      0.78    102994
weighted avg       0.77      0.77      0.77    102994

RandomForest model saved successfully at RandomForest_model.pkl.

Training XGBoost...
Evaluating XGBoost...
              precision    recall  f1-score   support

           0       0.92      0.92      0.92     18778
           1       0.95      0.95      0.95     20048
           2       0.97      0.99      0.98     18912
           3       0.59      0.58      0.58     23738
           4       0.55      0.56      0.56     21518

    accuracy            